### Лабораторная работа №1

**Данные для заданий 1 и 4 вам сообщит преподаватель**

1.	Любым удобным вам образом получите TLE аппарата.

ИСО, NOAA 18

In [1]:
# NOAA 18
# 1 28654U 05018A   23051.78784836  .00000288  00000+0  17890-3 0  9996
# 2 28654  98.9209 126.7023 0013919 353.4119   6.6867 14.12867280915135
# https://isstracker.pl/en/satelity/28654
from skyfield.api import EarthSatellite
from skyfield.api import load, wgs84
line1 = '1 28654U 05018A   23051.78784836  .00000288  00000+0  17890-3 0  9996'
line2 = '2 28654  98.9209 126.7023 0013919 353.4119   6.6867 14.12867280915135'
ts = load.timescale()
satellite = EarthSatellite(line1, line2, 'NOAA 18', ts)

2.	За положение антенны-наблюдателя возьмите координаты ЛК.

In [2]:
from skyfield.api import N,S,E,W
# Широта и долгода Долгопы, взятая с yandex maps
LK = wgs84.latlon(55.930171 * N, 37.518219 * W, elevation_m=172)

3. Интересуемый временной диапазон:

In [3]:
from datetime import datetime, timedelta

start, end = datetime.now(), datetime.now() + timedelta(days=1)

print("Интересуемый диапазон: c {} до {}".format(start.strftime("%d.%m.%Y %H:%M:%S"), end.strftime("%d.%m.%Y %H:%M:%S")))

Интересуемый диапазон: c 21.02.2023 17:55:26 до 22.02.2023 17:55:26


**Если вам требуются даты в другом формате - конвертацию делать НЕ запрещено**

4. Рассчитайте положение аппарата в указанной системе координат в заданном временном диапазоне.

In [11]:
from skyfield import api
import datetime
from datetime import datetime, timedelta
import pytz
from skyfield.api import utc

nummins = 24 * 60
base = datetime.now()
base = base.replace(tzinfo=utc)
date_list = [base + timedelta(minutes=x) for x in range(nummins)]
ts.from_datetimes(date_list)

positions = []
lats = []
lons = []
heights = []
for s in date_list:
    time = ts.from_datetime(s)
    geocentric = satellite.at(time) # Position
    lat, lon = wgs84.latlon_of(geocentric)
    height =   wgs84.height_of(geocentric)
    positions.append(geocentric)
    lats.append(lat)
    lons.append(lon)
    heights.append(height) 

5. Отобразите получившиеся координаты.

In [10]:
for g in positions:
    print('Position relative to ECI(km):', g.position.km)
    lat, lon = wgs84.latlon_of(g)
    height =   wgs84.height_of(g)
    print('Latitude and longitude respectively:', lat, lon)
    print('Height (km):', height.km)
    print()

Position relative to ECI(km): [ 3918.67626413 -5694.95720943 -2156.73273526]
Latitude and longitude respectively: -17deg 21' 08.5" -115deg 33' 36.7"
Height (km): 865.3028426613894

Position relative to ECI(km): [ 3777.11560952 -5618.01976502 -2571.30337994]
Latitude and longitude respectively: -20deg 50' 26.1" -116deg 25' 16.6"
Height (km): 866.1227261222915

Position relative to ECI(km): [ 3621.28019298 -5519.85023923 -2976.13200955]
Latitude and longitude respectively: -24deg 19' 28.9" -117deg 18' 39.8"
Height (km): 866.9999913806586

Position relative to ECI(km): [ 3451.76241987 -5400.82490948 -3369.68764861]
Latitude and longitude respectively: -27deg 48' 14.3" -118deg 14' 11.9"
Height (km): 867.9223170821675

Position relative to ECI(km): [ 3269.20616414 -5261.39877475 -3750.48286249]
Latitude and longitude respectively: -31deg 16' 39.4" -119deg 12' 23.3"
Height (km): 868.8762238137506

Position relative to ECI(km): [ 3074.30423952 -5102.10369506 -4117.07933158]
Latitude and longi

Position relative to ECI(km): [-4369.48859789  4707.65111537 -3305.72626052]
Latitude and longitude respectively: -27deg 27' 28.5" 141deg 53' 54.8"
Height (km): 850.1097255248417

Position relative to ECI(km): [-4439.88346697  4898.98356549 -2909.22303834]
Latitude and longitude respectively: -23deg 57' 38.7" 140deg 58' 25.1"
Height (km): 848.6759197922086

Position relative to ECI(km): [-4493.37511983  5071.66502514 -2501.6160521 ]
Latitude and longitude respectively: -20deg 27' 29.8" 140deg 05' 02.4"
Height (km): 847.3584439883351

Position relative to ECI(km): [-4529.75243445  5225.02963141 -2084.45685119]
Latitude and longitude respectively: -16deg 57' 04.8" 139deg 13' 21.1"
Height (km): 846.1714110928259

Position relative to ECI(km): [-4548.87000778  5358.48534024 -1659.33515048]
Latitude and longitude respectively: -13deg 26' 26.6" 138deg 22' 59.5"
Height (km): 845.1272686071765

Position relative to ECI(km): [-4550.64886841  5471.51641602 -1227.87263347]
Latitude and longitude 

6. Получите прогноз видимости аппарата для антенны-наблюдателя (то есть посчитайте и выведите в виде таблицы, в какие диапазоны времени антенна «видит» аппарат).

In [68]:
for s in date_list:
    time = ts.from_datetime(s)
    ICRS_LK = LK.at(time)

t0 = ts.now()
t1 = ts.now() + 1
t, events = satellite.find_events(LK, t0, t1, altitude_degrees=0.0)
# print(t[::2].utc_strftime('%Y %b %d %H:%M:%S'))
rises =   []
plunges = []
riseFlag = False
riseTime = 0
for ti, event in zip(t, events):
    if event == 0:
        riseFlag = True
        riseTime = ti
    elif event == 2:
        riseFlag = False
        plunges.append(t)
    else: continue
    name = ('rise above 0°', 'culminate', 'set below 0°')[event]
    #print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)
    #print()
    #print(rises)
for r, p in zip(rises, plunges):
    #print(r.utc_strftime('%Y %b %d %H:%M:%S'))
    #print(p.utc_strftime('%Y %b %d %H:%M:%S'))
    #print()
    #print(p[0], r[-1])
    duration = p[0] - r[-1]
    print(duration)

-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132
-0.9324337793514132


7. В промежутки времени, когда аппарат «виден» для антенны, посчитайте требуемые для связи углы азимута и элевации в каждый момент времени.

In [52]:
difference = satellite - LK    # Relative position obtained via vector substraction
topocentric = difference.at(ts.now()) # Access this position as plain coords.

alt, az, distance = topocentric.altaz() # Altitude & Azimuth

print('Altitude:', alt)
print('Azimuth:', az)
print(f'Distance: {distance.km} km')

Altitude: -53deg 42' 12.0"
Azimuth: 40deg 08' 24.3"
Distance: 11304.906437765274 km


8. Отобразите на графике в полярных координатах азимут-элевация получившиеся углы.<br>Полярным радиусом в вашей системе будет угол элевации (90 градусов по элевации - центр полярной сетки).

In [8]:
# Ваш код